<a href="https://colab.research.google.com/github/JoeOlang/Drafts/blob/master/Data%20Structures/Graphs/demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
###############################################################################
##     IMPORT PACKAGES     ####################################################
###############################################################################

import re

###############################################################################
##     HELPER FUNCTIONS     ###################################################
###############################################################################

def get_empty_appearances(characters, num = False):

    '''Create dictionary of dictionaries to track coappearances of characters

    Parameters
    ----------
    characters: list of character names

    num       : whether or not the dictionary of dictionaries should initialize
                to 0 (True) or an empty list (False)

    Returns
    -------
    Dictionary of dictionaries where each key is each character and each value
    is a dictionary with all other characters as keys, and values as 0 or [].
    '''

    appearances = {}
    for character in characters:
        companions = {}
        for companion in characters:
            if companion != character:

                if num:
                    companions[companion] = 0

                else:
                    companions[companion] = []
        appearances[character] = companions
    return appearances


def switch_char(old_char, new_char, lst):

    '''Switch character name in list

    Parameters
    ----------
    old_char: character name to be replaced

    new_char: character name to replace with

    lst     : list of character names

    Returns
    -------
    A list of character names with the old_char replaced with the new_char
    '''

    new_lst = [char for char in lst]
    new_lst.remove(old_char)

    # Check for existing duplicate
    if new_char not in new_lst:
        new_lst.extend([new_char])
    return new_lst

###############################################################################
##     ADD COAPPEARANCE FUNCTION     ##########################################
###############################################################################

def add_coappear(curr_lst, entered_lst, appearances, scene, internal_cast):

    '''Add coappearances between those who entered and those who are in the scene

    Parameters
    ----------
    curr_lst     : characters currently in the scene

    entered_lst  : characters that just entered

    appearances  : dictionary of dictionaries to store appearances

    scene        : scene number

    internal_cast: dictionary connecting cast to cast in play within a play

    Returns
    -------
    An updated dictionary of dictionaries with added coappearances
    '''

    # Create new lists for char_names (entered) and current_chars
    # Fairy = Fairies
    # Lords/Attendants = Attendants
    current_chars = [char for char in curr_lst]
    char_names    = [char for char in entered_lst]

    char_dict     = {'Fairy': 'Fairies',
                     'Lords': 'Attendants'}

    # Adjust names in current_chars and char_names (entered) for Fairies/Lords
    for char_key in char_dict.keys():
        if char_key in current_chars:
            current_chars = switch_char(char_key, char_dict[char_key],
                                        current_chars)
        if char_key in char_names:
            char_names    = switch_char(char_key, char_dict[char_key],
                                        char_names)

    # Adjust names in current_chars and char_names (entered) for internal cast
    for char_key in internal_cast.keys():
        if char_key in current_chars:
            current_chars = switch_char(char_key, internal_cast[char_key],
                                        current_chars)
        if char_key in char_names:
            char_names    = switch_char(char_key, internal_cast[char_key],
                                        char_names)

    # Handle Train, which can be Attendants or Fairies
    if 'Train' in char_names:
        if ('Oberon' in char_names) | ('Titania' in char_names):
            char_names = switch_char('Train', 'Fairies', char_names)

        elif ('Egeus' in char_names):
            char_names = switch_char('Train', 'Attendants', char_names)

    # Add co-appearances with each other
    for char in char_names:
        if char in appearances.keys():
            for co_char in char_names:
                if (co_char != char) & (co_char in appearances.keys()):
                    if scene not in appearances[char][co_char]:
                        appearances[char][co_char].extend([scene])

    # Add co-appearances with characters already in scene
    if len(current_chars) != 0:
        for char in current_chars:
            if char in appearances.keys():
                for co_char in char_names:
                    if (co_char != char) & (co_char in appearances.keys()):

                        # Check for duplicates
                        if scene not in appearances[char][co_char]:
                            appearances[char][co_char].extend([scene])

                        if scene not in appearances[co_char][char]:
                            appearances[co_char][char].extend([scene])

    return appearances

###############################################################################
##     MAIN FUNCTION     ######################################################
###############################################################################

def get_coappear(scene, act, int_cast_dict, all_appearances):

    '''Get coappearances that occur throughout a given scene's text

    Parameters
    ----------
    scene          : scene text, starts with scene number (I., II., III., ...)

    act            : string for act number in Roman numerals (I, II, III, ...)

    int_cast_dict  : dictionary for cast of play within a play

    all_appearances: dictionary of dictionaries for coappearances between all
                     characters in the play

    Returns
    -------
    An updated dictionary of dictionaries with added coappearances
    '''

    # Get stage directions before characters enter
    intro = scene.split('Enter')[0].strip()

    # Find all entrances, exits, and sleeping
    all_results = re.findall('(Enter [A-Za-z,:; \n’]+\.|[A-Z]+\.|Exit [A-Za-z, \n]+\.|Exit.|Exeunt [A-Za-z, \n]+\.|Exeunt.|\[_.*[Ss]leep.*_])', scene)
    all_results = [result.strip('[_').strip('_]') for result in all_results]

    curr_char = []  # Characters currently in scene
    prev_speak = '' # Who is current speaker likely speaking to
    curr_speak = '' # Who is the current speaker, can track who is exiting
    asleep = []     # Characters currently asleep (cannot exeunt)

    for result in all_results:

        print(result)

        # CASE: Get the act + scene number
        if result in ['I.', 'II.']:
            if curr_char == []:
                scene = act + result.strip('.').lower()

                # CASE: sleeping characters being in scene already
                # Get character names, add to current character
                # and sleeping lists
                if 'asleep' in intro:

                    # SPECIAL CASE: Titania referred to as The Queen
                    if 'The Queen' in intro:
                        asleep.extend(['Titania'])
                        curr_char.extend(asleep)

                    # CASE: all other sleeping characters
                    else:
                        chars = re.findall('[A-Z][a-z]+', intro.split('\n')[-1].strip())
                        asleep.extend(chars)
                        all_appearances = add_coappear(curr_char, asleep,
                                                       all_appearances, scene,
                                                       int_cast_dict)
                        curr_char.extend(asleep)

            else:
                continue

        # CASE: Characters have entered
        # Get character names, add coappearances, update current character list
        elif 'Enter' in result:

            # Find all proper nouns (characters) in result
            chars = re.findall('[A-Z][a-z]+', result.strip('Enter').strip())

            # CASE: fairies
            if result == 'Enter four Fairies.':
                fairies = ['Peaseblossom', 'Cobweb', 'Moth', 'Mustardseed']
                all_appearances = add_coappear(curr_char, fairies,
                                               all_appearances, scene,
                                               int_cast_dict)
                curr_char.extend(fairies)

            # CASE: all other characters
            else:
                all_appearances = add_coappear(curr_char, chars,
                                               all_appearances, scene,
                                               int_cast_dict)
                curr_char.extend(chars)

        # CASE: someone is speaking
        elif re.match('[A-Z]+\.', result):

            # New speaker, save current speaker as previous speaker
            if curr_speak != '':
                prev_speak = curr_speak

            curr_speak = result.strip('.').title()

            # If they are speaking then they could not still be asleep
            # Update sleeping list
            if curr_speak in asleep:
                asleep.remove(curr_speak)

        # CASE: exit without specified character, assume is curr_speak
        elif result == 'Exit.':

            # CASE: actors in play within play exit
            if curr_speak not in curr_char:
                actor = ''
                if curr_speak in int_cast_dict.keys():
                    actor = int_cast_dict[curr_speak]
                    curr_char.remove(actor)

                else:
                    print('WE HAVE A PROBLEM HERE.')

            # CASE: all other characters, assume curr_speak exits
            else:
                curr_char.remove(curr_speak)

        # CASE: exeunt without specified characters, assume all conscious
        #       characters exit
        elif result == 'Exeunt.':
            to_exit = [char for char in curr_char if char not in asleep]

            for char in to_exit:
                curr_char.remove(char)

        # CASE: specified characters exit
        elif (('Exit' in result) | ('Exeunt' in result)) & ('all but' not in result):

            # CASE: someone exits and someone sleeps in same stage direction
            if 'sleep' in result:
                stage_dir = result.split('. ')

                for dir_ in stage_dir:

                    # First half of direction, get those who exit
                    if 'Exeunt' in dir_:
                        chars = re.findall('[A-Z][a-z]+',
                                dir_.strip('Exeunt').strip())
                        for char in chars:
                            if (char == 'Fairies'):
                                if 'Fairies' in curr_char:
                                    curr_char.remove('Fairies')
                                elif 'Train' in curr_char:
                                    curr_char.remove('Train')

                    # Get other part of direction where someone sleeps
                    elif 'sleep' in dir_:
                        chars = re.findall('[A-Z][a-z]+', dir_)
                        for char in chars:
                            asleep.extend([char])

            # CASE: all other exit cases
            # Find characters in stage direction, update current character list
            else:
                chars = re.findall('[A-Z][a-z]+',
                        result.strip('Exit').strip().strip('Exeunt').strip())

                for char in chars:

                    # Stage directions interchangeably refer to Fairies and Trains
                    if (char == 'Fairies'):
                        if 'Fairies' in curr_char:
                            curr_char.remove('Fairies')

                        elif 'Train' in curr_char:
                            curr_char.remove('Train')

                    # Stage directions will interchangeably refer to clowns
                    elif char == 'Clowns':
                        for actor in ['Flute', 'Snout', 'Starveling', 'Quince', 'Snug']:
                            if actor in curr_char:
                                curr_char.remove(actor)

                    # CASE: just because one attendant leaves does not mean all
                    #       attendants leave
                    elif char == 'Attendant':
                        continue

                    # CASE: all other exits, just remove the character
                    else:
                        curr_char.remove(char)

        # CASE: exeunt all but, remove all current characters except those in
        #       stage direction
        elif 'Exeunt all but' in result:
            chars = re.findall('[A-Z][a-z]+', result.strip('Exeunt all but').strip())
            curr_char = [char for char in curr_char if char in chars]

        # CASE: sleep without specified character, assumes current speaker sleeps
        elif ('Sleeps.' in result) | ('Lies down and sleeps.' in result):
            asleep.extend([curr_speak])

        # CASE: assume that they refers to current speaker and previous speaker
        elif 'They sleep' in result:
            asleep.extend([curr_speak])
            asleep.extend([prev_speak])

    return all_appearances

In [1]:
# Import packages for data cleaning
import numpy as np
import pandas as pd
import re # For finding specific strings in the text
# Import packages for data visualization
import plotly.offline as py
import plotly.graph_objects as go
import networkx as nx

In [3]:
# Load the data
f = open("sample_data/data.txt", "r")
whole_text = f.read()

In [5]:
scenes = whole_text.split('SCENE ')
print(len(scenes))

10


In [6]:
characters = re.findall('[A-Z]+[,;]',scenes[0])
characters = [name.strip(',').strip(';').title() for name in characters]
characters

['Theseus',
 'Hippolyta',
 'Egeus',
 'Hermia',
 'Helena',
 'Lysander',
 'Demetrius',
 'Philostrate',
 'Quince',
 'Snug',
 'Bottom',
 'Flute',
 'Snout',
 'Starveling',
 'Oberon',
 'Titania',
 'Puck',
 'Goodfellow',
 'Peaseblossom',
 'Cobweb',
 'Moth',
 'Mustardseed',
 'Pyramus',
 'Thisbe',
 'Wall',
 'Moonshine',
 'Lion']

In [7]:
characters.remove('Goodfellow') # Puck = Goodfellow
characters.append('Fairies')
characters.append('Attendants')
characters

['Theseus',
 'Hippolyta',
 'Egeus',
 'Hermia',
 'Helena',
 'Lysander',
 'Demetrius',
 'Philostrate',
 'Quince',
 'Snug',
 'Bottom',
 'Flute',
 'Snout',
 'Starveling',
 'Oberon',
 'Titania',
 'Puck',
 'Peaseblossom',
 'Cobweb',
 'Moth',
 'Mustardseed',
 'Pyramus',
 'Thisbe',
 'Wall',
 'Moonshine',
 'Lion',
 'Fairies',
 'Attendants']

In [16]:
appearances = get_empty_appearances(characters)

In [12]:
#print(scenes[1])

In [17]:
scenes[9] = scenes[9].split('***')[0]

In [18]:
internal_cast = {'Pyramus': 'Bottom', 
                 'Thisbe': 'Flute', 
                 'Wall': 'Snout', 
                 'Moonshine': 'Starveling', 
                 'Lion': 'Snug', 
                 'Prologue': 'Quince'}

In [19]:
appearances_Ii = get_coappear(scenes[1], 'I', internal_cast, appearances)
appearances_Iii = get_coappear(scenes[2], 'I', internal_cast, appearances_Ii)

appearances_IIi = get_coappear(scenes[3], 'II', internal_cast, appearances_Iii)
appearances_IIii = get_coappear(scenes[4], 'II', internal_cast, appearances_IIi)

appearances_IIIi = get_coappear(scenes[5], 'III', internal_cast, appearances_IIii)
appearances_IIIii = get_coappear(scenes[6], 'III', internal_cast, appearances_IIIi)

appearances_IVi = get_coappear(scenes[7], 'IV', internal_cast, appearances_IIIii)
appearances_IVii = get_coappear(scenes[8], 'IV', internal_cast, appearances_IVi)

all_appearances = get_coappear(scenes[9], 'V', internal_cast, appearances_IVii)

I.
Enter Theseus, Hippolyta, Philostrate and Attendants.
THESEUS.
HIPPOLYTA.
THESEUS.
Exit Philostrate.
Enter Egeus, Hermia, Lysander and Demetrius.
EGEUS.
THESEUS.
EGEUS.
THESEUS.
HERMIA.
THESEUS.
HERMIA.
THESEUS.
HERMIA.
THESEUS.
HERMIA.
THESEUS.
DEMETRIUS.
LYSANDER.
EGEUS.
LYSANDER.
THESEUS.
EGEUS.
Exeunt all but Lysander and Hermia.
LYSANDER.
HERMIA.
LYSANDER.
HERMIA.
LYSANDER.
HERMIA.
LYSANDER.
HERMIA.
LYSANDER.
HERMIA.
LYSANDER.
HERMIA.
LYSANDER.
Enter Helena.
HERMIA.
HELENA.
HERMIA.
HELENA.
HERMIA.
HELENA.
HERMIA.
HELENA.
HERMIA.
HELENA.
HERMIA.
LYSANDER.
HERMIA.
LYSANDER.
Exit Hermia.
Exit Lysander.
HELENA.
Exit Helena.
II.
Enter Quince, Snug, Bottom, Flute, Snout  and Starveling.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
FLUTE.
QUINCE.
FLUTE.
QUINCE.
FLUTE.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
STARVELING.
QUINCE.
QUINCE.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTOM.
QUINCE.
BOTTO

In [20]:
appearance_counts = get_empty_appearances(characters, True)
scene_counts = {}

In [21]:
# For each character that appears, get how many scenes the character appears in and
# how many times each pair of characters appears together
for character in all_appearances:
    scene_counts[character] = []
    for co_char in all_appearances[character]:
        appearance_counts[character][co_char] = len(all_appearances[character][co_char])
        scene_counts[character].extend(all_appearances[character][co_char])
        
    scene_counts[character] = len(set(scene_counts[character]))

In [22]:
appearance_counts

{'Attendants': {'Bottom': 2,
  'Cobweb': 0,
  'Demetrius': 3,
  'Egeus': 2,
  'Fairies': 0,
  'Flute': 1,
  'Helena': 2,
  'Hermia': 3,
  'Hippolyta': 3,
  'Lion': 0,
  'Lysander': 3,
  'Moonshine': 0,
  'Moth': 0,
  'Mustardseed': 0,
  'Oberon': 0,
  'Peaseblossom': 0,
  'Philostrate': 2,
  'Puck': 0,
  'Pyramus': 0,
  'Quince': 1,
  'Snout': 1,
  'Snug': 1,
  'Starveling': 1,
  'Theseus': 3,
  'Thisbe': 0,
  'Titania': 0,
  'Wall': 0},
 'Bottom': {'Attendants': 2,
  'Cobweb': 2,
  'Demetrius': 2,
  'Egeus': 1,
  'Fairies': 1,
  'Flute': 4,
  'Helena': 2,
  'Hermia': 2,
  'Hippolyta': 2,
  'Lion': 0,
  'Lysander': 2,
  'Moonshine': 0,
  'Moth': 2,
  'Mustardseed': 2,
  'Oberon': 1,
  'Peaseblossom': 2,
  'Philostrate': 1,
  'Puck': 2,
  'Pyramus': 0,
  'Quince': 4,
  'Snout': 4,
  'Snug': 4,
  'Starveling': 4,
  'Theseus': 2,
  'Thisbe': 0,
  'Titania': 2,
  'Wall': 0},
 'Cobweb': {'Attendants': 0,
  'Bottom': 2,
  'Demetrius': 1,
  'Egeus': 0,
  'Fairies': 1,
  'Flute': 0,
  'Helena'

In [23]:
scene_counts

{'Attendants': 3,
 'Bottom': 5,
 'Cobweb': 2,
 'Demetrius': 6,
 'Egeus': 2,
 'Fairies': 4,
 'Flute': 4,
 'Helena': 6,
 'Hermia': 5,
 'Hippolyta': 3,
 'Lion': 0,
 'Lysander': 5,
 'Moonshine': 0,
 'Moth': 2,
 'Mustardseed': 2,
 'Oberon': 5,
 'Peaseblossom': 2,
 'Philostrate': 2,
 'Puck': 6,
 'Pyramus': 0,
 'Quince': 4,
 'Snout': 4,
 'Snug': 4,
 'Starveling': 4,
 'Theseus': 3,
 'Thisbe': 0,
 'Titania': 5,
 'Wall': 0}

In [24]:
midsummer = nx.Graph()

In [25]:
# Add node for each character
for char in scene_counts.keys():
    if scene_counts[char] > 0:
        midsummer.add_node(char, size = scene_counts[char])

In [26]:
midsummer.nodes

NodeView(('Theseus', 'Hippolyta', 'Egeus', 'Hermia', 'Helena', 'Lysander', 'Demetrius', 'Philostrate', 'Quince', 'Snug', 'Bottom', 'Flute', 'Snout', 'Starveling', 'Oberon', 'Titania', 'Puck', 'Peaseblossom', 'Cobweb', 'Moth', 'Mustardseed', 'Fairies', 'Attendants'))

In [27]:
# For each co-appearance between two characters, add an edge
for char in appearance_counts.keys():
    for co_char in appearance_counts[char].keys():
        
        # Only add edge if the count is positive
        if appearance_counts[char][co_char] > 0:
            midsummer.add_edge(char, co_char, weight = appearance_counts[char][co_char])

In [28]:
midsummer.edges()

EdgeView([('Theseus', 'Hippolyta'), ('Theseus', 'Egeus'), ('Theseus', 'Hermia'), ('Theseus', 'Helena'), ('Theseus', 'Lysander'), ('Theseus', 'Demetrius'), ('Theseus', 'Philostrate'), ('Theseus', 'Quince'), ('Theseus', 'Snug'), ('Theseus', 'Bottom'), ('Theseus', 'Flute'), ('Theseus', 'Snout'), ('Theseus', 'Starveling'), ('Theseus', 'Attendants'), ('Hippolyta', 'Egeus'), ('Hippolyta', 'Hermia'), ('Hippolyta', 'Helena'), ('Hippolyta', 'Lysander'), ('Hippolyta', 'Demetrius'), ('Hippolyta', 'Philostrate'), ('Hippolyta', 'Quince'), ('Hippolyta', 'Snug'), ('Hippolyta', 'Bottom'), ('Hippolyta', 'Flute'), ('Hippolyta', 'Snout'), ('Hippolyta', 'Starveling'), ('Hippolyta', 'Attendants'), ('Egeus', 'Hermia'), ('Egeus', 'Helena'), ('Egeus', 'Lysander'), ('Egeus', 'Demetrius'), ('Egeus', 'Bottom'), ('Egeus', 'Attendants'), ('Hermia', 'Helena'), ('Hermia', 'Lysander'), ('Hermia', 'Demetrius'), ('Hermia', 'Philostrate'), ('Hermia', 'Quince'), ('Hermia', 'Snug'), ('Hermia', 'Bottom'), ('Hermia', 'Flute

In [29]:
# Get positions for the nodes in G
pos_ = nx.spring_layout(midsummer)

In [30]:
pos_

{'Attendants': array([-0.68631161, -0.10665588]),
 'Bottom': array([0.08592812, 0.17797978]),
 'Cobweb': array([ 0.3452703 , -0.87746124]),
 'Demetrius': array([-0.12629421, -0.31051032]),
 'Egeus': array([-0.90321482, -0.55939916]),
 'Fairies': array([ 0.89086043, -0.10445371]),
 'Flute': array([-0.4357781 ,  0.71167046]),
 'Helena': array([ 0.07511334, -0.14454083]),
 'Hermia': array([-0.23647415, -0.16282851]),
 'Hippolyta': array([-0.7176582,  0.2714235]),
 'Lysander': array([-0.10460006, -0.00709562]),
 'Moth': array([ 0.63517391, -0.85215282]),
 'Mustardseed': array([ 0.86411147, -0.65835538]),
 'Oberon': array([ 0.47469584, -0.47046691]),
 'Peaseblossom': array([ 1.        , -0.41314354]),
 'Philostrate': array([-0.94175004,  0.6055747 ]),
 'Puck': array([0.51812151, 0.1607699 ]),
 'Quince': array([-0.32028352,  0.50723451]),
 'Snout': array([0.10402912, 0.74605515]),
 'Snug': array([-0.2029336 ,  0.87047243]),
 'Starveling': array([-0.07474474,  0.64522253]),
 'Theseus': array(

In [32]:
midsummer.edges()[('Theseus', 'Egeus')]

{'weight': 2}

In [33]:
def make_edge(x, y, text, width):
    
    '''Creates a scatter trace for the edge between x's and y's with given width

    Parameters
    ----------
    x    : a tuple of the endpoints' x-coordinates in the form, tuple([x0, x1, None])
    
    y    : a tuple of the endpoints' y-coordinates in the form, tuple([y0, y1, None])
    
    width: the width of the line

    Returns
    -------
    An edge trace that goes between x0 and x1 with specified width.
    '''
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')

In [34]:
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in midsummer.edges():
    
    if midsummer.edges()[edge]['weight'] > 0:
        char_1 = edge[0]
        char_2 = edge[1]

        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]

        text   = char_1 + '--' + char_2 + ': ' + str(midsummer.edges()[edge]['weight'])
        
        trace  = make_edge([x0, x1, None], [y0, y1, None], text,
                           0.3*midsummer.edges()[edge]['weight']**1.75)

        edge_trace.append(trace)

In [35]:
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in midsummer.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([5*midsummer.nodes()[node]['size']])
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

In [36]:
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)


fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False)

fig.update_xaxes(showticklabels = False)

fig.update_yaxes(showticklabels = False)

fig.show()
py.plot(fig, filename='midsummer_network.html')

'midsummer_network.html'